# Add working conditions data from EWCS

### Load libraries

In [1]:
import pandas as pd
import pyreadstat

### Load data

In [2]:
df, meta = pyreadstat.read_dta(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/EWCS 1991-2015 UKDA ISCO.DTA"
)
variable_labels = meta.column_labels

In [3]:
df

,id,countid,wave,year,y15_Q1,y15_Q2a,y15_Q2b,y15_Q2c,y15_Q2d,y15_Q3a_2,...,wq_slim,goodsoc,envsec,intens,intens_slim,prosp,wlb,wlb_slim,ISCO_08,ISCO_88
0,BE000964,1,6,2015,1,1,40,1,2,NaN,...,14.285715,94.444443,66.666664,31.759258,53.333336,25.000000,69.12500,88.500,8141,8231
1,BE000967,1,6,2015,1,2,49,1,2,NaN,...,85.714287,NaN,91.025642,13.148149,24.444445,50.000000,84.50000,87.500,5141,5141
2,BE000968,1,6,2015,2,1,54,1,2,2,...,85.714287,NaN,76.923080,27.037039,52.222221,50.000000,56.31250,37.500,1323,1313
3,BE000970,1,6,2015,3,1,52,1,2,2,...,28.571430,97.916664,98.717949,9.444445,18.888889,62.500000,86.84375,96.875,7115,7124
4,BE000972,1,6,2015,3,1,28,1,2,1,...,42.857143,74.166664,97.435898,30.833334,53.333336,41.666664,74.28125,96.875,8322,8322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178900,21034,27,1,1991,5,1,36,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178901,21037,27,1,1991,3,2,32,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178902,21042,27,1,1991,4,1,17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178903,21045,27,1,1991,4,1,23,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Choose only necessary data, tranform where needed

Identify and name the aggregated indexes for work conditions

In [4]:
df = df.rename(
    columns={
        "adincome_mth": "jqi_monthly_earnings",
        "wq": "jqi_skills_discretion",
        # "goodsoc": "jqi_social_environment", - to calculate manually
        "envsec": "jqi_physical_environment",
        "intens_slim": "jqi_intensity",
        # "prosp": "jqi_prospects", - to calculate manually
        "wlb_slim": "jqi_working_time_quality",
    }
)

Define country names and choose the needed

In [5]:
countid_mapping = meta.value_labels["COUNTID"]
df["countid"] = df["countid"].map(countid_mapping)

In [6]:
countries = [
    "Austria",
    "Belgium",
    "Czech Republic",
    "Denmark",
    "Estonia",
    "France",
    "Germany",
    "Italy",
    "Slovenia",
    "Spain",
    "Switzerland",
]
df = df[df["countid"].isin(countries)].reset_index(drop=True)

Leave only waves 5 and 6 (2010 and 2015)

In [7]:
df = df[df.year >= 2010].reset_index(drop=True)

Drop lines with missing isco codes

In [8]:
df = df.dropna(subset="ISCO_08").reset_index(drop=True)

Adjust some isco codes

In [9]:
def modify_isco(value):
    if len(str(value)) == 1:
        return value * 1000
    elif len(str(value)) == 2:
        return value * 100
    elif len(str(value)) == 3:
        return value * 10
    else:
        return value


df["ISCO_08"] = df["ISCO_08"].apply(modify_isco)

Rename some variables

In [10]:
df = df.rename(columns={"countid": "country", "ISCO_08": "isco"})

Format id column

In [11]:
df["id"] = df["id"].astype(str)
df["id"] = df["id"].str.replace(r"[^ -~]+", "", regex=True)
df["id"] = df["id"].str.strip()

In [12]:
df

,id,country,wave,year,y15_Q1,y15_Q2a,y15_Q2b,y15_Q2c,y15_Q2d,y15_Q3a_2,...,wq_slim,goodsoc,jqi_physical_environment,intens,jqi_intensity,prosp,wlb,jqi_working_time_quality,isco,ISCO_88
0,BE000964,Belgium,6,2015,1,1,40,1,2,NaN,...,14.285715,94.444443,66.666664,31.759258,53.333336,25.000000,69.12500,88.500,8141,8231
1,BE000967,Belgium,6,2015,1,2,49,1,2,NaN,...,85.714287,NaN,91.025642,13.148149,24.444445,50.000000,84.50000,87.500,5141,5141
2,BE000968,Belgium,6,2015,2,1,54,1,2,2,...,85.714287,NaN,76.923080,27.037039,52.222221,50.000000,56.31250,37.500,1323,1313
3,BE000970,Belgium,6,2015,3,1,52,1,2,2,...,28.571430,97.916664,98.717949,9.444445,18.888889,62.500000,86.84375,96.875,7115,7124
4,BE000972,Belgium,6,2015,3,1,28,1,2,1,...,42.857143,74.166664,97.435898,30.833334,53.333336,41.666664,74.28125,96.875,8322,8322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34701,23000011509,Slovenia,5,2010,3,2,63,NaN,NaN,1,...,57.142860,NaN,84.615387,NaN,40.000000,NaN,NaN,100.000,9111,9131
34702,23000011511,Slovenia,5,2010,4,2,21,NaN,NaN,1,...,50.000000,NaN,84.615387,NaN,40.000000,NaN,NaN,62.500,5131,5123
34703,23000011513,Slovenia,5,2010,2,2,48,NaN,NaN,1,...,71.428574,NaN,80.769234,NaN,6.666667,NaN,NaN,87.500,5141,5141
34704,23000011516,Slovenia,5,2010,5,1,41,NaN,NaN,1,...,100.000000,NaN,97.435898,NaN,18.888889,NaN,NaN,88.500,3113,3113


### Calculate social environment index

Load data for 2010 and 2015

In [13]:
df10, meta10 = pyreadstat.read_dta(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/UKDA-2010/stata/stata11/ewcs_2010_version_ukda_6_dec_2011.dta"
)

Calculate the index for 2010

In [14]:
soc10 = df10[
    [
        "id",
        "q70a",
        "q70b",
        "q70c",
        "q71a",
        "q71c",
        "q71b",
        "q58b",
        "q58a",
        "q51a",
        "q51b",
    ]
]
soc10 = soc10.dropna().reset_index(drop=True)
for col in soc10.columns:
    if col != "id":
        soc10[col] = soc10[col].astype("int")
        soc10 = soc10[soc10[col] < 7].reset_index(drop=True)
soc10[["q58a", "q58b"]] = soc10[["q58a", "q58b"]].replace({1: 2, 2: 1})
soc10[["q51a", "q51b"]] = soc10[["q51a", "q51b"]].replace({1: 5, 2: 4, 4: 2, 5: 1})

In [15]:
soc10.describe()

,id,q70a,q70b,q70c,q71a,q71c,q71b,q58b,q58a,q51a,q51b
count,3.083600e+04,30836.000000,30836.000000,30836.000000,30836.000000,30836.000000,30836.000000,30836.000000,30836.000000,30836.000000,30836.000000
mean,1.572380e+10,1.891069,1.982196,1.945875,1.980834,1.991017,1.953528,1.950123,1.805682,4.017220,3.759826
std,1.017743e+10,0.311558,0.132240,0.226268,0.137110,0.094354,0.210508,0.217694,0.395681,1.012743,1.164070
min,1.000001e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,6.000005e+09,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000
50%,1.503820e+10,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,4.000000,4.000000
75%,2.500000e+10,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,5.000000,5.000000
max,3.400018e+10,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,5.000000,5.000000


In [16]:
soc10["jqi_social_environment"] = (
    soc10["q58a"]
    + soc10["q58b"]
    + soc10["q51a"]
    + soc10["q51b"]
    + soc10["q70a"]
    + soc10["q70b"]
    + soc10["q70c"]
    + soc10["q71a"]
    + soc10["q71b"]
    + soc10["q71c"]
)

old_min = 10
old_max = 26
new_min = 0
new_max = 100

soc10["jqi_social_environment"] = (
    (soc10["jqi_social_environment"] - old_min) / (old_max - old_min)
) * (new_max - new_min) + new_min

In [17]:
soc10["jqi_social_environment"].describe()

count    30836.000000
mean        82.983566
std         14.530474
min         12.500000
25%         75.000000
50%         87.500000
75%         93.750000
max        100.000000
Name: jqi_social_environment, dtype: float64

In [18]:
soc10 = soc10[["id", "jqi_social_environment"]]
soc10["year"] = 2010

Calculate the index for 2015

In [19]:
soc15 = df[
    [
        "id",
        "y15_Q80a",
        "y15_Q80b",
        "y15_Q80c",
        "y15_Q81a",
        "y15_Q81b",
        "y15_Q81c",
        "y15_Q63a",
        "y15_Q63e",
        "y15_Q61a",
        "y15_Q61b",
    ]
]
soc15 = soc15.dropna().reset_index(drop=True)
for col in soc15.columns:
    if col != "id":
        soc15[col] = soc15[col].astype("int")
        soc15 = soc15[soc15[col] < 7].reset_index(drop=True)
soc15[["y15_Q61a", "y15_Q61b", "y15_Q63a", "y15_Q63e"]] = soc15[
    ["y15_Q61a", "y15_Q61b", "y15_Q63a", "y15_Q63e"]
].replace({1: 5, 2: 4, 4: 2, 5: 1})

In [20]:
soc15.describe()

,y15_Q80a,y15_Q80b,y15_Q80c,y15_Q81a,y15_Q81b,y15_Q81c,y15_Q63a,y15_Q63e,y15_Q61a,y15_Q61b
count,13279.000000,13279.000000,13279.000000,13279.000000,13279.000000,13279.000000,13279.000000,13279.000000,13279.000000,13279.000000
mean,1.872430,1.980194,1.950072,1.980420,1.990737,1.941637,4.426538,3.817682,3.967543,3.644100
std,0.333623,0.139337,0.217805,0.138556,0.095800,0.234437,0.855332,1.163330,1.079077,1.223853
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,4.000000,3.000000,3.000000,3.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,5.000000,4.000000,4.000000,4.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,5.000000,5.000000,5.000000,5.000000
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,5.000000,5.000000,5.000000,5.000000


In [21]:
soc15["jqi_social_environment"] = (
    soc15["y15_Q80a"]
    + soc15["y15_Q80b"]
    + soc15["y15_Q80c"]
    + soc15["y15_Q81a"]
    + soc15["y15_Q81b"]
    + soc15["y15_Q81c"]
    + soc15["y15_Q63a"]
    + soc15["y15_Q63e"]
    + soc15["y15_Q61a"]
    + soc15["y15_Q61b"]
)

old_min = 10
old_max = 32
new_min = 0
new_max = 100

soc15["jqi_social_environment"] = (
    (soc15["jqi_social_environment"] - old_min) / (old_max - old_min)
) * (new_max - new_min) + new_min

In [22]:
soc15["jqi_social_environment"].describe()

count    13279.000000
mean        79.869788
std         15.768143
min          9.090909
25%         72.727273
50%         81.818182
75%         90.909091
max        100.000000
Name: jqi_social_environment, dtype: float64

In [23]:
soc15 = soc15[["id", "jqi_social_environment"]]
soc15["year"] = 2015

Add indexes to main df

In [24]:
soc = pd.concat([soc10, soc15], axis=0).reset_index(drop=True)

In [25]:
soc["id"] = soc["id"].astype(str)
soc["id"] = soc["id"].astype(str).apply(lambda x: x[:-2] if x.endswith(".0") else x)
soc["id"] = soc["id"].str.replace(r"[^ -~]+", "", regex=True)

In [26]:
df = df.merge(soc, on=["id", "year"], how="left")

In [27]:
unique_mergeid_df = set(df["id"].unique())
unique_mergeid_soc = set(soc["id"].unique())
intersection_ids = unique_mergeid_df.intersection(unique_mergeid_soc)
len(intersection_ids)

25750

### Calculate prospects index

Index for 2010

In [28]:
pro10 = df10[["id", "q77c", "q77a"]]
pro10 = pro10.dropna().reset_index(drop=True)
for col in pro10.columns:
    if col != "id":
        pro10[col] = pro10[col].astype("int")
        pro10 = pro10[pro10[col] < 7].reset_index(drop=True)
pro10["q77c"] = pro10["q77c"].replace({1: 5, 2: 4, 4: 2, 5: 1})

In [29]:
pro10.describe()

,id,q77c,q77a
count,3.855600e+04,38556.000000,38556.000000
mean,1.610812e+10,3.254254,2.293340
std,1.032083e+10,1.199690,1.226394
min,1.000001e+09,1.000000,1.000000
25%,7.000102e+09,2.000000,1.000000
50%,1.538090e+10,3.000000,2.000000
75%,2.600020e+10,4.000000,3.000000
max,3.400018e+10,5.000000,5.000000


In [30]:
pro10["jqi_prospects"] = pro10["q77c"] + pro10["q77a"]

old_min = 2
old_max = 10
new_min = 0
new_max = 100

pro10["jqi_prospects"] = ((pro10["jqi_prospects"] - old_min) / (old_max - old_min)) * (
    new_max - new_min
) + new_min

In [31]:
pro10["jqi_prospects"].describe()

count    38556.000000
mean        44.344914
std         23.024819
min          0.000000
25%         25.000000
50%         50.000000
75%         62.500000
max        100.000000
Name: jqi_prospects, dtype: float64

In [32]:
pro10 = pro10[["id", "jqi_prospects"]]
pro10["year"] = 2010

Index for 2015

In [33]:
pro15 = df[["id", "y15_Q89b", "y15_Q89g"]]
pro15 = pro15.dropna().reset_index(drop=True)
for col in pro15.columns:
    if col != "id":
        pro15[col] = pro15[col].astype("int")
        pro15 = pro15[pro15[col] < 7].reset_index(drop=True)
pro15["y15_Q89g"] = pro15["y15_Q89g"].replace({1: 5, 2: 4, 4: 2, 5: 1})

In [34]:
pro15.describe()

,y15_Q89b,y15_Q89g
count,30073.000000,30073.000000
mean,3.202374,2.159445
std,1.282885,1.270620
min,1.000000,1.000000
25%,2.000000,1.000000
50%,3.000000,2.000000
75%,4.000000,3.000000
max,5.000000,5.000000


In [35]:
pro15["jqi_prospects"] = pro15["y15_Q89b"] + pro15["y15_Q89g"]

old_min = 2
old_max = 10
new_min = 0
new_max = 100

pro15["jqi_prospects"] = ((pro15["jqi_prospects"] - old_min) / (old_max - old_min)) * (
    new_max - new_min
) + new_min

In [36]:
pro15["jqi_prospects"].describe()

count    30073.000000
mean        42.022745
std         24.018895
min          0.000000
25%         25.000000
50%         37.500000
75%         50.000000
max        100.000000
Name: jqi_prospects, dtype: float64

In [37]:
pro15 = pro15[["id", "jqi_prospects"]]
pro15["year"] = 2015

Add indexes to main df

In [38]:
pro = pd.concat([pro10, pro15], axis=0).reset_index(drop=True)

In [39]:
pro["id"] = pro["id"].astype(str)
pro["id"] = pro["id"].astype(str).apply(lambda x: x[:-2] if x.endswith(".0") else x)
pro["id"] = pro["id"].str.replace(r"[^ -~]+", "", regex=True)

In [40]:
df = df.merge(pro, on=["id", "year"], how="left")

In [41]:
unique_mergeid_df = set(df["id"].unique())
unique_mergeid_pro = set(pro["id"].unique())
intersection_ids = unique_mergeid_df.intersection(unique_mergeid_pro)
len(intersection_ids)

30073

### Add country level post-stratification weights

In [42]:
columns_to_multiply = [
    "jqi_monthly_earnings",
    "jqi_skills_discretion",
    "jqi_social_environment",
    "jqi_physical_environment",
    "jqi_intensity",
    "jqi_prospects",
    "jqi_working_time_quality",
]

for column in columns_to_multiply:
    df[f"{column}_weighted"] = df[column] * df["w4"]

### Aggregate on the level of isco, year and country

In [43]:
# Choose columns
df = df[
    [
        "country",
        "year",
        "isco",
        "jqi_monthly_earnings",
        "jqi_skills_discretion",
        "jqi_social_environment",
        "jqi_physical_environment",
        "jqi_intensity",
        "jqi_prospects",
        "jqi_working_time_quality",
        "jqi_monthly_earnings_weighted",
        "jqi_skills_discretion_weighted",
        "jqi_social_environment_weighted",
        "jqi_physical_environment_weighted",
        "jqi_intensity_weighted",
        "jqi_prospects_weighted",
        "jqi_working_time_quality_weighted",
    ]
]

In [44]:
df = df.groupby(["country", "year", "isco"]).mean().reset_index(drop=False)

In [45]:
df = df.dropna().reset_index(drop=True)

Interpolate to obtain approximation of values for 2011 and 2013

In [46]:
df["year"] = pd.to_datetime(df["year"], format="%Y")

df_2010 = df[df["year"].dt.year == 2010]
df_2015 = df[df["year"].dt.year == 2015]

df_2011 = pd.DataFrame()
df_2013 = pd.DataFrame()

for country in df["country"].unique():
    for isco in df["isco"].unique():
        data_2010 = df_2010[(df_2010["country"] == country) & (df_2010["isco"] == isco)]
        data_2015 = df_2015[(df_2015["country"] == country) & (df_2015["isco"] == isco)]

        if not data_2010.empty and not data_2015.empty:
            interpolated_values_2011 = data_2010.iloc[0, 3:].interpolate(
                method="linear", limit_area="inside", limit=1
            ) + (data_2015.iloc[0, 3:] - data_2010.iloc[0, 3:]).multiply(1 / 3)

            interpolated_values_2013 = data_2010.iloc[0, 3:].interpolate(
                method="linear", limit_area="inside", limit=1
            ) + (data_2015.iloc[0, 3:] - data_2010.iloc[0, 3:]).multiply(3 / 5)

            interpolated_values_2011.index = data_2010.columns[3:]
            interpolated_values_2013.index = data_2010.columns[3:]

            row_2011 = pd.DataFrame(
                {
                    "country": [country],
                    "year": [pd.Timestamp(year=2011, month=1, day=1)],
                    "isco": [isco],
                }
            ).join(interpolated_values_2011.to_frame().T.reset_index(drop=True))

            row_2013 = pd.DataFrame(
                {
                    "country": [country],
                    "year": [pd.Timestamp(year=2013, month=1, day=1)],
                    "isco": [isco],
                }
            ).join(interpolated_values_2013.to_frame().T.reset_index(drop=True))

            df_2011 = pd.concat([df_2011, row_2011], ignore_index=True)
            df_2013 = pd.concat([df_2013, row_2013], ignore_index=True)

df = pd.concat([df, df_2011, df_2013], ignore_index=True)
df["year"] = pd.to_datetime(df["year"], format="%Y")
df["year"] = df["year"].dt.year

/var/folders/ck/3npmnx597nb730qdy5w_6kp00000gn/T/ipykernel_62548/814221703.py:15: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_values_2011 = data_2010.iloc[0, 3:].interpolate(
/var/folders/ck/3npmnx597nb730qdy5w_6kp00000gn/T/ipykernel_62548/814221703.py:19: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_values_2013 = data_2010.iloc[0, 3:].interpolate(
/var/folders/ck/3npmnx597nb730qdy5w_6kp00000gn/T/ipykernel_62548/814221703.py:15: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  interpolated_values_2011 = data_2010.iloc[0, 3:].interpolate(
/var/folders/ck/3npmnx597nb730qdy5w_6kp00000gn/T/ipykernel_62

In [47]:
df

,country,year,isco,jqi_monthly_earnings,jqi_skills_discretion,jqi_social_environment,jqi_physical_environment,jqi_intensity,jqi_prospects,jqi_working_time_quality,jqi_monthly_earnings_weighted,jqi_skills_discretion_weighted,jqi_social_environment_weighted,jqi_physical_environment_weighted,jqi_intensity_weighted,jqi_prospects_weighted,jqi_working_time_quality_weighted
0,Austria,2010,1114,2366.148112,77.290234,100.0,97.863248,45.925926,25.0,100.0,3034.577207,94.40666,60.31181,121.392346,67.171461,15.077952,123.996761
1,Austria,2010,1120,3143.596761,83.616787,87.5,85.042736,50.74074,21.428571,53.125,1638.952541,52.464895,47.019517,53.989173,30.725478,14.436884,32.531523
2,Austria,2010,1219,1588.023401,64.30635,93.75,89.903848,25.694444,40.625,85.15625,1166.245217,44.854093,82.620009,60.938954,16.828854,29.519498,59.592109
3,Austria,2010,1221,3194.299927,86.929178,82.8125,95.512821,57.77778,25.0,85.15625,2370.625566,66.103378,63.071333,72.55156,42.98721,18.639903,65.149279
4,Austria,2010,1321,2788.674561,66.906092,81.25,72.863248,48.611112,25.0,60.0625,2632.194486,96.736029,97.872116,105.282028,67.344148,32.894051,101.218176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7186,Spain,2013,9520,863.823664,50.285447,87.840909,76.007326,18.0,49.375,91.917857,834.09199,42.925449,80.752616,62.740049,15.72124,35.272711,77.438312
7187,Spain,2013,2100,2069.535937,66.338245,97.5,88.46154,49.999997,42.5,98.75,3654.135875,115.455341,170.467285,154.932298,116.419693,68.796142,171.49274
7188,Spain,2013,3258,1435.902368,67.310957,96.363636,71.28205,40.222222,42.5,68.7,1456.858411,72.508648,100.578868,76.626674,39.327916,44.635262,70.518447
7189,Spain,2013,7533,617.640436,43.9013,90.227273,82.365386,47.916668,63.75,83.921875,440.003997,45.112082,66.129333,83.0141,44.046678,48.808201,70.344564


In [48]:
df.to_csv(
    "/Users/alexandralugova/Documents/GitHub/MH-old-workers/data/datasets/work_quality_indexes.csv",
    index=False,
)